# Predicting Movie Revenue

## Features  

**Variable to Predict:** Revenue


- Budget  
- Runtime  
- Genre  
- Release Date  
 - Season  
 - Holiday   
- Production Companies  
 - Number of companies involved  
 - Number of movies a company has made (previously)  
   - Histogram of how often top 10% of production companies make movies
- Crew
 - Number of crewmembers  
- Cast
 - Gender Ratio for top 2/5/10/25
 - Number of previous movies top 10 cast have been in
- Collection
 - Is sequel  


In [1]:
import pandas as pd
import sqlite3

In [2]:
db = sqlite3.connect(r'../../../Data Science Data/Unit 3/db.sqlite')

In [15]:
## SQL import statements to create dataframes

# Base dataframe to join other onto
base_df = pd.read_sql("""SELECT movie_id, budget, release_date, revenue, runtime, imdb_budget, imdb_revenue, title 
                         FROM movies 
                         WHERE (budget > 0 OR imdb_budget > 0) AND (revenue > 0 OR imdb_revenue > 0)
                      """, db, index_col='movie_id')
base_df.index = base_df.index.map(int)

# Pull in genre table to create dummy variables
genre_df = pd.read_sql("""SELECT * FROM genres""", db)

# Create series for number of production companies associated with each movie_id
number_of_prod_companies = pd.read_sql("""SELECT movie_id, COUNT(company_name)
                                          FROM production_companies
                                          GROUP BY 1
                                          """, db, index_col='movie_id')

# Create series for number of crew members associated with each movie_id
number_of_crewmembers = pd.read_sql("""SELECT movie_id, COUNT(crew_member_id)
                                       FROM crew
                                       GROUP BY 1
                                       """, db, index_col='movie_id')

# Dataframe for cast data to create gender ratio and previous movies for cast features 
cast_data = pd.read_sql("""SELECT cc.movie_id, cc.cast_member_id, cc.cast_order, c.name, c.gender, m.release_date
                           FROM cast_credit cc
                           JOIN cast c ON cc.cast_member_id = c.cast_member_id
                           JOIN movies m ON m.movie_id = cc.movie_id
                           """, db, index_col = 'movie_id')

TypeError: read_sql() missing 2 required positional arguments: 'sql' and 'con'

In [14]:
pivot_genre = genre_df.pivot_table(index='movie_id',columns='genre_name', values='genre_name',aggfunc=len, fill_value=0)
pivot_genre.index = pivot_genre.index.map(int)
pivot_genre = pivot_genre.apply(pd.to_numeric)

In [13]:
df = base_df.join(pivot_genre, how='inner')
df = df.join(number_of_prod_companies, how='inner')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6557 entries, 819 to 84165
Data columns (total 27 columns):
budget             6557 non-null int64
release_date       6557 non-null object
revenue            6557 non-null int64
runtime            6546 non-null float64
imdb_budget        2096 non-null object
imdb_revenue       2175 non-null object
title              6557 non-null object
Action             6557 non-null int64
Adventure          6557 non-null int64
Animation          6557 non-null int64
Comedy             6557 non-null int64
Crime              6557 non-null int64
Documentary        6557 non-null int64
Drama              6557 non-null int64
Family             6557 non-null int64
Fantasy            6557 non-null int64
Foreign            6557 non-null int64
History            6557 non-null int64
Horror             6557 non-null int64
Music              6557 non-null int64
Mystery            6557 non-null int64
Romance            6557 non-null int64
Science Fiction    6557 no